<a href="https://colab.research.google.com/github/matthew-e-thomas/Plant_Image_Project/blob/main/model_experiments_Plant_Image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Import Libraries, get paths ready, and clone PlantDoc Repo

In [ ]:
import numpy as np
import tensorflow as tf
import IPython.display as display
print(tf.__version__)

2.3.0


In [ ]:
!python --version

Python 3.6.9


In [ ]:
!git clone https://github.com/matthew-e-thomas/PlantDoc-Dataset.git

Cloning into 'PlantDoc-Dataset'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 2661 (delta 18), reused 3 (delta 1), pack-reused 2628
Receiving objects: 100% (2661/2661), 932.25 MiB | 44.41 MiB/s, done.
Resolving deltas: 100% (19/19), done.
Checking out files: 100% (2578/2578), done.


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#run this if cloning plantdoc github
from pathlib import Path
data_dir = Path.cwd() / 'PlantDoc-Dataset'

In [ ]:
train_dir = data_dir.joinpath('train')
test_dir = data_dir.joinpath('test')

In [ ]:
#CLASS_NAMES_TRAIN = [item.name for item in train_dir.glob('*')]

In [ ]:
#run this if using improved plantdoc dataset from your google drive
# import os
# os.chdir('/content/drive/My Drive')

In [ ]:
#!unzip PlantDoc.v1-resize-416x416.tfrecord.zip

Archive:  PlantDoc.v1-resize-416x416.tfrecord.zip
 extracting: train/leaves.tfrecord   
 extracting: train/leaves_label_map.pbtxt  
 extracting: test/leaves.tfrecord    
 extracting: test/leaves_label_map.pbtxt  
 extracting: README.roboflow.txt     
 extracting: README.dataset.txt      


In [ ]:
# filepaths = ["train/leaves.tfrecord"]
# dataset = tf.data.TFRecordDataset(filepaths)
# for item in dataset.take(5):
#     print(item)

In [ ]:
raw_image_dataset = tf.data.TFRecordDataset(['train/leaves.tfrecord'])

# Create a dictionary describing the features.
image_feature_description = {
    #'height': tf.io.FixedLenFeature([], tf.int64),
    #'width': tf.io.FixedLenFeature([], tf.int64),
    #'depth': tf.io.FixedLenFeature([], tf.int64),
    'image': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.string),
}

def _parse_image_function(example_proto):
  # Parse the input tf.train.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)

parsed_image_dataset = raw_image_dataset.map(_parse_image_function)
parsed_image_dataset

<MapDataset shapes: {label: ()}, types: {label: tf.string}>

In [ ]:
# for image_features in parsed_image_dataset.take(5):
#   image_raw = image_features['image'].numpy()
#   display.display(display.Image(data=image_raw))

### Create Train, Validation, and Test Set

In [ ]:
batch_size = 32
# img_height = 150
# img_width = 150
img_height = 224
img_width = 224

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_dir,
  validation_split=0.1,
  subset="training",
  seed=123,
  #label_mode="categorical",
  #class_names=CLASS_NAMES,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 2340 files belonging to 27 classes.
Using 2106 files for training.


In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_dir,
  validation_split=0.1,
  subset="validation",
  seed=123,
  #label_mode="categorical",
  #class_names=CLASS_NAMES,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 2340 files belonging to 27 classes.
Using 234 files for validation.


In [ ]:
# class names
class_names = train_ds.class_names
print(class_names)
print(len(class_names))

['Apple Scab Leaf', 'Apple leaf', 'Apple rust leaf', 'Bell_pepper leaf', 'Bell_pepper leaf spot', 'Blueberry leaf', 'Cherry leaf', 'Corn Gray leaf spot', 'Corn leaf blight', 'Corn rust leaf', 'Peach leaf', 'Potato leaf early blight', 'Potato leaf late blight', 'Raspberry leaf', 'Soyabean leaf', 'Squash Powdery mildew leaf', 'Strawberry leaf', 'Tomato Early blight leaf', 'Tomato Septoria leaf spot', 'Tomato leaf', 'Tomato leaf bacterial spot', 'Tomato leaf late blight', 'Tomato leaf mosaic virus', 'Tomato leaf yellow virus', 'Tomato mold leaf', 'grape leaf', 'grape leaf black rot']
27


In [ ]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  test_dir,
  seed=123,
  #label_mode="categorical",
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 236 files belonging to 27 classes.


### Create and Train Model

In [ ]:
from tensorflow import keras
from tensorflow.keras.applications import inception_resnet_v2

In [ ]:
from keras.applications import InceptionResNetV2

In [ ]:
def preprocess(image, label):
    #resized_image = tf.image.resize(image, [224, 224])
    final_image = inception_resnet_v2.preprocess_input(image)
    return final_image, label

In [ ]:

train_set = train_ds.map(preprocess).prefetch(1)
valid_set = val_ds.map(preprocess).prefetch(1)
test_set = test_ds.map(preprocess).prefetch(1)

In [ ]:
# from tensorflow import keras
# from tensorflow.keras import layers
# from tensorflow.keras.models import Sequential

# # data augmentation
# data_augmentation = keras.Sequential(
#   [
#     layers.experimental.preprocessing.RandomFlip("horizontal", 
#                                                  input_shape=(img_height, 
#                                                               img_width,
#                                                               3)),
#     layers.experimental.preprocessing.RandomRotation(0.1),
#     layers.experimental.preprocessing.RandomZoom(0.1),
#   ]
# )



In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

monitor_val_acc = EarlyStopping(monitor = 'val_loss', patience=15)

In [ ]:
tf.random.set_seed(123)
np.random.seed(123)
base_model = InceptionResNetV2(weights="imagenet", include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
#x = keras.layers.Dense(1024, activation='relu')(avg)
output = keras.layers.Dense(len(class_names), activation="softmax")(avg)
model = keras.models.Model(inputs=base_model.input, outputs=output)

219062272/219055592 [==============================] - 2s 0us/step


In [ ]:
adagrad = tf.keras.optimizers.Adagrad()
nadam = tf.keras.optimizers.Nadam(.0001)

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer=nadam, metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=40, callbacks=monitor_val_acc)
score = model.evaluate(test_set)

Epoch 1/40
66/66 [==============================] - 51s 768ms/step - loss: 2.3866 - accuracy: 0.3253 - val_loss: 1.7386 - val_accuracy: 0.4872
Epoch 2/40
66/66 [==============================] - 47s 707ms/step - loss: 0.8774 - accuracy: 0.7555 - val_loss: 1.4046 - val_accuracy: 0.5726
Epoch 3/40
66/66 [==============================] - 47s 706ms/step - loss: 0.2728 - accuracy: 0.9406 - val_loss: 1.3900 - val_accuracy: 0.6026
Epoch 4/40
66/66 [==============================] - 47s 706ms/step - loss: 0.1058 - accuracy: 0.9777 - val_loss: 1.4348 - val_accuracy: 0.6154
Epoch 5/40
66/66 [==============================] - 46s 704ms/step - loss: 0.0764 - accuracy: 0.9815 - val_loss: 1.4967 - val_accuracy: 0.6368
Epoch 6/40
66/66 [==============================] - 47s 707ms/step - loss: 0.0572 - accuracy: 0.9805 - val_loss: 1.6304 - val_accuracy: 0.5769
Epoch 7/40
66/66 [==============================] - 47s 707ms/step - loss: 0.0634 - accuracy: 0.9796 - val_loss: 1.6540 - val_accuracy: 0.5940

In [ ]:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

0 input_6
1 conv2d_1015
2 batch_normalization_1015
3 activation_1015
4 conv2d_1016
5 batch_normalization_1016
6 activation_1016
7 conv2d_1017
8 batch_normalization_1017
9 activation_1017
10 max_pooling2d_20
11 conv2d_1018
12 batch_normalization_1018
13 activation_1018
14 conv2d_1019
15 batch_normalization_1019
16 activation_1019
17 max_pooling2d_21
18 conv2d_1023
19 batch_normalization_1023
20 activation_1023
21 conv2d_1021
22 conv2d_1024
23 batch_normalization_1021
24 batch_normalization_1024
25 activation_1021
26 activation_1024
27 average_pooling2d_5
28 conv2d_1020
29 conv2d_1022
30 conv2d_1025
31 conv2d_1026
32 batch_normalization_1020
33 batch_normalization_1022
34 batch_normalization_1025
35 batch_normalization_1026
36 activation_1020
37 activation_1022
38 activation_1025
39 activation_1026
40 mixed_5b
41 conv2d_1030
42 batch_normalization_1030
43 activation_1030
44 conv2d_1028
45 conv2d_1031
46 batch_normalization_1028
47 batch_normalization_1031
48 activation_1028
49 activation

In [ ]:
for layer in model.layers[:673]:
   layer.trainable = False
for layer in model.layers[673:]:
   layer.trainable = True

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer=nadam, metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=35, callbacks=monitor_val_acc)
score = model.evaluate(test_set)

Epoch 1/35
66/66 [==============================] - 36s 539ms/step - loss: 0.0552 - accuracy: 0.9905 - val_loss: 1.6927 - val_accuracy: 0.6282
Epoch 2/35
66/66 [==============================] - 33s 494ms/step - loss: 0.0345 - accuracy: 0.9910 - val_loss: 1.7198 - val_accuracy: 0.6154
Epoch 3/35
66/66 [==============================] - 33s 493ms/step - loss: 0.0280 - accuracy: 0.9929 - val_loss: 1.6906 - val_accuracy: 0.6325
Epoch 4/35
66/66 [==============================] - 32s 490ms/step - loss: 0.0155 - accuracy: 0.9934 - val_loss: 1.7205 - val_accuracy: 0.6282
Epoch 5/35
66/66 [==============================] - 32s 490ms/step - loss: 0.0128 - accuracy: 0.9948 - val_loss: 1.7054 - val_accuracy: 0.6368
Epoch 6/35
66/66 [==============================] - 32s 489ms/step - loss: 0.0128 - accuracy: 0.9948 - val_loss: 1.7155 - val_accuracy: 0.6325
Epoch 7/35
66/66 [==============================] - 32s 490ms/step - loss: 0.0100 - accuracy: 0.9957 - val_loss: 1.7561 - val_accuracy: 0.6325

In [ ]:
#model.save('/content/drive/My Drive/plant_inceptionresnet_model')
model.save('/content/drive/My Drive/plant_inceptionresnet_model_h5.h5')

## Try Image Augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
def preprocess_gen(image):
    #resized_image = tf.image.resize(image, [224, 224])
    final_image = inception_resnet_v2.preprocess_input(image)
    return final_image

In [ ]:
train_datagen = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        shear_range=0.2,
        zoom_range=0.2,
        preprocessing_function=preprocess_gen)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_gen)
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(299, 299),
        batch_size=32)
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(299, 299),
        batch_size=32)

Found 2340 images belonging to 27 classes.
Found 236 images belonging to 27 classes.


In [ ]:
model.compile(loss="categorical_crossentropy", optimizer=nadam, metrics=["accuracy"])

In [ ]:
history = model.fit(train_generator, epochs=22)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/22
74/74 [==============================] - 113s 2s/step - loss: 0.6175 - accuracy: 0.8150
Epoch 2/22
74/74 [==============================] - 108s 1s/step - loss: 0.3664 - accuracy: 0.8885
Epoch 3/22
74/74 [==============================] - 107s 1s/step - loss: 0.2479 - accuracy: 0.9282
Epoch 4/22
74/74 [==============================] - 106s 1s/step - loss: 0.2250 - accuracy: 0.9291
Epoch 5/22
74/74 [==============================] - 107s 1s/step - loss: 0.1518 - accuracy: 0.9487
Epoch 6/22
74/74 [==============================] - 108s 1s/step - loss: 0.1482 - accuracy: 0.9564
Epoch 7/22
74/74 [==============================] - 107s 1s/step - loss: 0.1122 - accuracy: 0.9650
Epoch 8/22
74/74 [==============================] - 107s 1s/step - loss: 0.1096 - accuracy: 0.9671
Epoch 9/22
74/74 [==============================] - 107s 1s/step - loss: 0.0944 - accuracy: 0.9718
Epoch 10/22
74/74 [==============================] - 107s 1s/step - loss: 0.1082 - accuracy: 0.9645
Epoch 11/

In [ ]:
model.evaluate(test_generator)

8/8 [==============================] - 7s 871ms/step - loss: 2.4283 - accuracy: 0.5720


[2.4282925128936768, 0.5720338821411133]

### Xception Model

In [ ]:
from keras.applications import Xception

In [ ]:
base_model = Xception(include_top=False, 
                               weights='imagenet', 
                               pooling='avg')

output = keras.layers.Dense(len(class_names), activation='softmax')(base_model.output)
model = keras.models.Model(inputs=base_model.inputs, outputs=output)
model.compile(
    loss="sparse_categorical_crossentropy", 
    optimizer="nadam", 
    metrics=["accuracy"])
history = model.fit(train_ds, epochs=10)
score = model.evaluate(test_ds)

Epoch 1/10
59/59 [==============================] - 43s 721ms/step - loss: 2.1950 - accuracy: 0.3570
Epoch 2/10
59/59 [==============================] - 42s 716ms/step - loss: 1.0706 - accuracy: 0.6601
Epoch 3/10
59/59 [==============================] - 43s 727ms/step - loss: 0.6377 - accuracy: 0.8063
Epoch 4/10
59/59 [==============================] - 43s 731ms/step - loss: 0.3776 - accuracy: 0.8895
Epoch 5/10
59/59 [==============================] - 42s 715ms/step - loss: 0.3023 - accuracy: 0.9088
Epoch 6/10
59/59 [==============================] - 43s 734ms/step - loss: 0.3845 - accuracy: 0.8847
Epoch 7/10
59/59 [==============================] - 42s 718ms/step - loss: 0.2582 - accuracy: 0.9146
Epoch 8/10
59/59 [==============================] - 42s 708ms/step - loss: 0.2441 - accuracy: 0.9216
Epoch 9/10
59/59 [==============================] - 42s 713ms/step - loss: 0.1586 - accuracy: 0.9472
Epoch 10/10
8/8 [==============================] - 1s 109ms/step - loss: 4.0204 - accuracy: